<a href="https://colab.research.google.com/github/Yutongzhang20080108/Using-a-Combination-of-DPO-and-RAG-to-Improve-the-Chinese-Poetry-Generation/blob/main/Test_between_Human_and_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 615.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_openai import ChatOpenAI

generate_model = ChatOpenAI(model="gpt-3.5-turbo", base_url="https://api.chatanywhere.tech/v1")
grade_model = ChatOpenAI(model="gpt-4o-ca", base_url="https://api.chatanywhere.tech/v1")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
generate_system_template = "你是一个善于写中国古代诗歌的诗人，接下来我会给你提供一个题目，请你围绕这个题目写一首古诗。在生成的过程中请你注意中国古代诗歌的格式要求，并且请你表达尽可能多的情感与含义。在最后输出结果时只需要生成诗歌的主体部分"
generate_prompt_template = ChatPromptTemplate.from_messages(
    [("system", generate_system_template), ("user", "{text}")]
)

grade_system_template = "你是一个中国古代诗歌评论家，接下来我会给你两首诗歌，请你依据以下标准来评价这两首诗歌的得分。评分标准为：①格式。生成的唐诗是否遵循写作规则,尤其注意对仗、句数、字数、平仄（0至5分）；②创新性。生成的唐诗是否抄袭现成的诗句，是否具有创新性（0至5分）；③相关性。生成的唐诗与给定主题是否相关（0至5分）；④美学。生成的唐诗是否符合中国诗词的审美，包括寓情于物等（0至5分）；⑤整体。从整体情况评价生成的唐诗（0至10分）.请你在最后给出这几项的得分，并且你不需要给出评判的过程和任何文字只需要输出最后的5个数字。"
grade_prompt_template = ChatPromptTemplate.from_messages(
    [("system", grade_system_template), ("user", "{text}")]
)

In [ ]:
generate_chain = generate_prompt_template | generate_model | parser
grade_chain = grade_prompt_template | grade_model | parser

In [ ]:
import json
with open("/content/唐诗三百首.json", encoding="utf-8") as f:
  original_poetry = json.load(f)
#print(original_poetry[0])
original_text = []
for each in original_poetry:
  text = each["paragraphs"]
  text_all = ""
  for one in text:
    text_all = text_all + one
  original_text.append(text_all)

FileNotFoundError: [Errno 2] No such file or directory: '/content/唐诗三百首.json'

In [ ]:
pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
from opencc import OpenCC
cc = OpenCC('t2s')
def convert(text):
  converted = cc.convert(text)
  return convert

In [ ]:
original_text_converted = []
for each in original_text:
  one = cc.convert(each)
  original_text_converted.append(one)

In [ ]:
theme = []
for each in original_poetry:
  title = each["title"]
  theme.append(cc.convert(title))

In [ ]:
generated_text = []
generated_score = []
for i in range(100):
   poem = generate_chain.invoke(f"请你围绕{theme[i]}这个题目写一首古诗")
   grade = grade_chain.invoke(f"{poem}")
   generated_text.append(poem)
   generated_score.append(grade)
print(generated_text)
print(generated_score)

['登高望远思难穹，\\\n岳咏蝉鸣动心中。\\\n夜深人静听声断，\\\n悠扬婉转入梦中。', '幽州台高远，登临望秋色。\n烟波浩渺间，思绪如飞絮。\n山川皆画境，云霞共舞曲。\n心随风飘荡，何处是归宿。', '昔日先贤孔子明，经邹鲁祭感怀情。\n百年风雨传佳话，千古遗风仰圣贤。\n凝思往事泪沾袖，铭记功德心永远。\n教化苍生传不朽，千古丰碑永流传。', '青袍红袖舞将进，醉卧花间思绪深。\n琴韵悠扬酒尚温，一曲高歌情不禁。\n金樽对酒心似火，豪情万丈意何极。\n风吹杨柳醉眼瞳，浮生若梦醉中寻。', '黄河远上白云间，横吹曲辞出塞关。\n万里戈声吹戍急，孤城孤驿望乡关。\n胡尘暮色笼平野，战马踏雪渡寒山。\n忆君心事殷勤问，愿将边疆报平安。', '黄河曲曲出塞地，胡马横行万里间。\n烽火台边烽烟起，将军独立望乡关。\n沙场战鼓齐鸣响，刀枪如雨血染山。\n忆往昔，边烽火；思归人，泪沾巾。\n今朝岁月今何在，烽火犹燃塞外天。\n愿得安邦归故里，不再横吹曲辞传。', '关山月，横吹曲辞，\n独倚栏杆思往事。\n寒风吹散忧愁意，\n皎皎明月照我心。\n\n山川万里情无限，\n江湖风波梦自深。\n且将心事随风去，\n任月光洒满山阴。', '婚礼喜临门，新嫁娘踏红被。\n花烛辉煌夜，笑语绕耳畔。\n亲朋皆祝福，良人情意切。\n共结良缘牢，白首不分离。', '故行宫，古木参天碧瓦红，\n往事如烟，落霞映照残阳。\n殿宇荒凉，风吹落叶满庭廊，\n寂寞凝望，燕归无人诉离伤。\n往昔繁华，今成空谷幽长，\n凝望远方，泪洒断桥残梦乡。', '锦瑟悠扬古调清，缠绵低吟入夜情。\n弦断音绝泪沾袖，翩跹舞动思绪生。\n长夜寂寞凝霜雪，孤灯照影泪漫流。\n梦中锦瑟如花落，凄凉离恨断肠愁。', '碧树枝头唤晓蝉，清风徐来叶欲翻。\n声声绕梁思无穷，千古传颂韵犹存。\n岁月流转似轮转，何惧芳华转眼繁。\n但愿人间能长在，与蝉共舞世间恩。', '青山翠竹绕乐游，溪水潺潺草萋萋。\n林间鸟语韵琴韵，花间风起舞幽幽。\n登高一望云蒸霞，心随飞鸟意无羁。\n且将闲逸留心间，与山水共乐游行。', '夜深人静雨潺潺，\n寄情北国思难言。\n孤灯一盏照窗外，\n泪滴无声滋心田。\n\n江水东流似故人，\n相逢何处共悲辛。\n愁云渡口难相见，\n独守寒窗待明晨。', '韩碑高耸入云天，\n千古传扬史载传。\n铭

In [ ]:
human_score = []
for i in range(100):
  grade = grade_chain.invoke(f"{original_text_converted[i]}")
  human_score.append(grade)
print(human_score)

['4, 4, 4, 4, 8', '5 5 5 5 10', '4 4 4 4 8', '5 2 5 5 8', '5 5 5 5 10', '5 5 4 5 9', '5 5 5 5 9', '3.5, 3.5, 3, 4, 7.5', '5 5 5 5 10', '5 5 4 5 9', '4 4 3 4 8', '5 5 5 5 10', '5 5 5 5 10', '5 5 5 5 10', '4, 4, 5, 4, 8', '4 4 4 4 8', '5 5 5 5 10', '4 4 3 4 8', '5 5 4 5 9', '4.5, 5, 5, 4.5, 9', '4,5,4,5,8', '4 4 5 4 8', '5 5 5 5 10', '4 4 3 4 8', '5 4.5 4 4.5 8.5', '4, 4, 5, 4, 8', '5 4 5 5 9', '5, 5, 5, 5, 9', '4, 5, 3, 4, 7', '5 5 4 5 9', '4.5 4.0 4.0 4.5 8.5', '4 3 4 4 8', '4,4,4,4,8', '4, 4, 3, 4, 8', '4.5 4 4.5 4.5 9', '4 5 4 4 8', '4,4,4,4,8', '5 5 4 5 9', '4 4 4 4 8', '4,4,4,4,8', '4 4 5 4 8', '4, 4, 5, 4, 8', '4 4 3 3 7', '4 4 4 4 8', '5 4 4 4 8', '4, 3, 3, 4, 7', '5 4 4 5 8', '5 5 5 5 9', '4 3 4 4 8', '5 5 5 5 10', '5 5 5 5 10', '5 4 4 5 9', '4 4 4 4 8', '5 5 5 5 10', '5 4 4 4 8', '5 4 5 5 9', '5,4,5,4,8', '5 4 5 4 8', '5 5 5 5 9', '4 4 3 4 8', '4 4 4 4 8', '5 4 4 5 8', '5 5 4 5 9', '4 4 3 4 7', '4 4 3 4 7', '5 5 5 4 9', '4, 5, 4, 4, 7', '4 4 4 4 8', '4.5 4.5 4 4.5 8.5', '4,4,3,

In [ ]:
def data_processing(data_path):
  with open(data_path, "r", encoding="utf-8") as f:
    result = json.load(f)

  cleaned_result = []
  for each in result:
    cleaned_text = each.replace('\n', '')
    cleaned_result.append(cleaned_text)
  return cleaned_result

In [ ]:
def grade(text):
  grade_result = []
  for each in text:
    grade = grade_chain.invoke(f"{each}")
    grade_result.append(grade)
  return grade_result

In [ ]:
score = grade(result)
print(score)

['4 4 4 4 8', '4 4 4 4 8', '4, 4, 3, 4, 7', '4 4 3 4 7', '5 4 5 5 8', '4 4 4 4 8', '4 5 4 4 8', '4 4 3 3 7', '4.5 4 4.5 4 8.5', '4 4 3 4 7', '4 4 3 3 7', '4,4,4,4,8', '4, 4, 3, 4, 7', '4 4 3 4 7', '5 5 5 5 10', '4 4 4 4 8', '4 5 4 4 8', '4.5, 4, 3.5, 4.5, 8.5', '4 4 3 4 7', '5 4 4 5 8', '5 4 4 4 8', '5 4 5 5 9', '5 4 3 4 7', '4,4,3,4,7', '4 4 3 4 7', '4 4 5 4 8', '4.5, 4, 4.5, 4.5, 8.5', '4 4 3 4 8', '4 4 4 4 8', '4 4 3 4 8', '4 4 4 4 8', '4 4 3 4 7', '4 4 4 4 8', '4, 3, 3, 3, 6', '5 5 5 5 10', '4,4,3,4,8', '4, 5, 3, 4, 8', '5 4 4 5 9', '4,4,4,4,8', '4 4 4 4 8', '4 4 4 4 8', '5 5 5 5 10', '4, 4, 4, 4, 8', '3 3 3 3 6', '5 5 4 4 8', '5 5 4 5 9', '4 4 4 4 8', '4 4 3 4 7', '4 4 3 4 7', '4 5 4 4 8', '5, 4, 4, 5, 9', '4 4 4 4 8', '5 5 4 5 9', '4 4 4 4 8', '4 4 4 5 8', '4.5 4.5 4.5 4.5 9', '4 4 4 4 7', '4 4 3 4 7', '4 4 3 4 7', '3 4 3 4 7', '5 4 5 5 9', '4 4 3 4 7', '4 5 3 4 7', '4 5 4 4 8', '4, 3, 3, 4, 7', '4, 4, 5, 4, 8', '4, 4, 3, 4, 7', '3 4 3 4 7', '3 4 2 3 6', '5 5 5 5 10', '5 4 4 4 8'

In [ ]:
def pipeline(theme_list):
  result = []
  grade_result = []
  for each in theme_list:
    poem = generate_chain.invoke(f"请你围绕{each}这个题目写一首古诗")
    result.append(poem)
  for each in result:
    grade = grade_chain.invoke(f"{each}")
    grade_result.append(grade)
  return result, grade_result

In [ ]:
themes = [
    "青春", "梦想", "高山", "深海", "星空", "失落", "爱情", "孤独", "希望", "迷茫",
    "成长", "离别", "重逢", "夕阳", "清晨", "冬雪", "春花", "夏雨", "秋叶", "往事",
    "忧伤", "欢乐", "旅行", "归宿", "朋友", "家庭", "传统", "创新", "自由", "约束",
    "城市", "乡村", "河流", "山谷", "荒漠", "林间", "夜晚", "黎明", "阴雨", "晴天",
    "风景", "人群", "孤单", "热闹", "平静", "激情", "压力", "解脱", "寻找", "发现",
    "失去", "获得", "旧时光", "新生", "悲伤", "喜悦", "分手", "团聚", "哲理", "纷争",
    "和平", "冒险", "安全", "学习", "教育", "成就", "失败", "胜利", "竞争", "合作",
    "梦魇", "幻想", "现实", "逃避", "面对", "变化", "恒常", "追忆", "遗忘", "诗歌",
    "绘画", "音乐", "舞蹈", "雕塑", "建筑", "文学", "历史", "未来", "科技", "自然",
    "灾害", "恢复", "衰退", "繁荣", "衰弱", "强健", "疾病", "治愈", "传说", "神话"
]
poem, score = pipeline(themes)
print(poem)
print(score)

['春风吹动青丝发，  \n芳草萋萋绿满园。  \n少年如花娇艳丽，  \n笑靥如花动人魂。  \n\n梦想披荆斩棘路，  \n热血沸腾勇向前。  \n岁月匆匆逝如箭，  \n青春永驻心头田。', '梦中花开映月华，\n一念飞扬入云霞。\n翩翩羽翼逐风起，\n逍遥天地逸尘沙。\n\n凌波微步登仙境，\n丹青妙手绘神华。\n若问梦想何所在，\n心中自有长安花。', '青峰耸立云霄间，\n巍峨高山入画图。\n挺拔峰峦云雾里，\n千古传颂壮丽愈。\n\n巨石奇峰立云天，\n峻岭奇峰耸碧烟。\n苍松翠柏摇风韵，\n山水相连意气鲜。', '深海茫茫波浪阔，\n浮光跃金鳞翩跹。\n深情莫测似幽梦，\n浩渺无涯藏玄机。\n\n潜游鲸鲵动轻盈，\n涌泉飞珠溅涟漪。\n蓝色苍茫心荡漾，\n谁能洞悉其中味。', '繁星点点银河横，\n夜幕降临掩昼光。\n寂静宇宙无边际，\n漫天繁星耀眼中。\n\n流萤点点飞舞间，\n思绪如云难安宁。\n仰望苍穹心悸动，\n星空璀璨梦中行。', '寂寞深夜失落情，\n孤灯孤枕泪两行。\n忆往昔恩爱如水，\n如今空留断肠声。\n\n风吹残梦入悲伤，\n心事难诉悔难当。\n花落花开皆匆匆，\n人间离合总难长。', '红颜昔日共谁知，\n相携并肩心相随。\n风吹花落泪珠滴，\n情深意重恨无疑。\n\n山水无情情自长，\n爱恨情仇皆虚妄。\n但愿相伴到白头，\n此生此世情不凉。', '独立寒江水，孤舟泛夕阳。\n寂寞思无限，孤灯照空房。\n独坐听风雨，孤云随飘忙。\n孤独何所恃，唯有心声长。', '明月高悬挂枝头，\n繁星点缀夜空愁。\n独倚栏杆思无尽，\n只愿人间皆如初。', '迷茫心绪难平静，烟雨蒙蒙映古城。\n柳岸花间听杜鹃，思绪飘飘恍如梦。\n人生若只如初见，何须纷扰乱心弦。\n行云流水任往来，迷茫追寻自心间。', '青春岁月似流水，成长之路漫漫长。\n蓦然回首思往事，曾是稚嫩少年郎。\n勤奋苦学逐梦想，携手挑战风浪涛。\n历经坎坷锤炼心，方知人生不易忘。\n\n青山不老绿还青，成长之路自有情。\n莫待时光匆匆逝，珍惜当下莫虚耽。\n扬帆远航逐梦想，勇往直前无畏惶。\n终将功成名不负，青春如歌笑颜开。', '暮色苍茫忆故人，离别情长泪沾巾。\n相思万缕难言语，断肠人在远山春。', '碧水横流共白鹭，重逢之际泪交绪。\n前尘往事随风散，今日相见喜悦无。\n

In [ ]:
poem = [
    "未老莫还乡 青山隐隐水茫茫 前度刘郎今又来 芳草小楼人未归",
    "自昔怀严濑，今为爱禹门。风生万木暮，月照一帆还。客路经江国，归心忆帝乡。无因接高论，空此望飞霜。",
    "万岁声长在，千峰影不移。  \n汉文思贾谊，应降便风期。",
    "万顷洪波总是愁。",
    "九万六千日 与子同遨游  \n日月掷人去 如此来何求",
    "一自玉京游赏后 十年不醉武陵春",
    "何处结同心。西陵松柏下。",
    "独坐南轩月皎洁，西南方隅旦霞发。  \n起行绕庭了却眠，天窗未明复幽独。",
    "日月东西驰不息，万物与天皆历数。  \n但知塞北草新绿，不觉江南花已故。  \n少年行路今耐得，白发还乡定无处。  \n欲将归计报君恩，未向边头防虏骑。",
    "独有苍头鸟，山栖不羡笼。向人空旧识，无伴苦相从。晚下松门北，归来竹径重。自怜行役者，辛苦未能终。",
    "少壮不努力 老大徒伤悲 人生非金石 不可永年 弃置勿复陈 天道终相报",
    "长卿牢落悲空舍，曼倩诙谐取自容。  \n见尔定还思校尉，梦归时复到临邛。  \n病来把酒不知厌，老去从人却自觉。  \n料得明年春晚日，北山还拟一相从。",
    "白发悲明镜，青春换故林。  \n山行愁向晚，江路恐生沈。  \n客思逢春尽，乡程隔岁寻。  \n几时同旧识，日夕醉金簪。",
    "万里秋空里，天高白日低。 晚来何处望，山色近人齐。",
    "几处早莺争暖树，谁家新燕啄春泥。  \n乱花渐欲迷人眼，浅草才能没马蹄。",
    "千门皆若水，万室尽成冰。日色明寒玉，风声静夜cken。  \n暗随明月去，远逐太阳还。莫道春来好，还愁春去残。",
    "几处落英香尚在，一川明月冷于铅。  \n故人独向城南去，不待黄鹂四月天。",
    "独眠秋夜夜，凉月向人明。  \n露下天如白，风生树有声。  \n病多无处避，贫极不关情。  \n自是归山计，年来未得行。",
    "曾逐东风拂舞筵，乐游春苑断肠天。  \n如何肯到清秋日，已带斜阳又带蝉。",
    "当时一别各天涯，今日相逢是几秋。  \n却话少年俱老大，只应长笑镜中头。",
    "何以静幽忧，松竹常自好。  \n万籁皆已寂，孤琴亦无调。  \n如何拂危弦，坐听哀鸿叫。",
    "绿酒清歌同尽兴，朱门画戟莫留宾。  \n主人不把金貂换，归路应随白日人。",
    "万里西风入战尘，关河犹拥旧将军。  \n飘零未到长安日，摇落空馀白发人。",
    "独夜他乡泪，年年为客愁。  \n已知无定所，非是不回头。",
    "白首相知犹按剑，朱门多怨即移棋。  \n相逢尽道休官好，未必能胜散吏时。",
    "四海无远图，一门有深隐。  \n我来值秋潦，已复过江浔。  \n独有终南山，白云空向寝。",
    "莫笑田家老瓦盆，半生随分不妨深。  \n佳人玉指排新雪，稚子金盆盛落英。  \n山舍春残花渐少，水乡寒尽日偏阴。  \n晚来一滴何曾有，独向松梢挂夕金。",
    "日月几万岁。与君书好音。",
    "天马出蒲梢兮凤鸟来昆吾， 银鞍绣被厌朝沽。  \n五湖四海有贤主，肯使风尘长弊余。",
    "汉帝重阿娇 乃令下井观 人生失意处 莫若古所叹 置酒长安道 声价倾五都 但令一顾重 飞步青云端 却思洛中日 倾倒白玉樽 何由见清影 空此对明君 严城闭百二 歌吹满三台 虽有梁甫吟 长当从尔回",
    "万里投荒已自哀，忍看山色更江台。  \n愁窥白发羞微禄，病凿黄丛厌手栽。  \n旅梦乱随帆过海，乡书长为雁传回。  \n遥知拜庆日，应醉曲江头。",
    "山泽无晨征 [1] ，林端送残月。  \n行子不顾反 [2] ，边人空结袜 [3] 。  \n白水照横关 [4] ，清渠来滑辙 [5] 。  \n秋草南陌头 [6] ，归心千里折 [7] 。",
    "地肺天池共一涯，不疑乘兴是生涯。  \n夜添星影沙边月，朝助帆阴竹里霞。  \n海阔有时同鼓枻，路长无处问分车。  \n知君别后应相忆，更想连峰在晚花。",
    "自是无心学变红。更将何物比衰翁。  \n晚来一饷风吹雨，恰似平生几阵风。",
    "西塞云深驿路长，北风吹雨度严阳。  \n只今何处逢春酒，满眼胡沙暗夕阳。",
    "独入林间路 无人知处深",
    "独怜京国人南窜，不似湘江水北流。",
    "独有残灯在，相看不自禁。",
    "萧条秋雨里，独出倚层台。  \n万井人何在，千山鸟不回。  \n孤云犹故态，衰草更空来。  \n落日西风起，苍茫暮色开。",
    "晓色明秦苑 皇家旧帝台 紫宸三月望 雨后一花开 野客来方醉 深云去不回 谁能骑马出 遥听落金枚",
    "偶向高台见，分明似旧游。山川连北固，风日得东州。独鹤归华表，双凫出汉洲。更怜春色在，先入谢公楼。",
    "千门尚如昔，万姓已非初。日月无偏照，山河不共居。空馀长乐殿，犹对建章壖。更想前朝事，萧条尽梦之。",
    "何处多风雨，南山有故园。旧山唯在梦，寒露独沾衣。白发今逾老，清时不易违。知君访林虑，何日更相归。",
    "三月正当三十日，风光别我苦吟身。  \n共知人事无远近，俱是一春行处尘。",
    "静中兼物象，静里得心源。日月三秋近，江湖万里开。夜深松露滴，时有乱禽来。若使诸侯见，应须换草莱。",
    "落日驻行殿 青年当此期 万方今仰德 一德可齐夷",
    "平生管城子，心事付毫发。## 贫病瘦如鹤，无人知此价。## 穷巷秋雨中，闭门无往客。## 带甲满天地，胡为君索客。## 一饭四五起，安能稽古册。## 长歌击唾壶，行坐白日没。## 感此岁云暮，寒云夜飞雪。",
    "空门寂寂淡吾身，溪雨微微洗客尘。  \n卧向白云情未尽，任他黄鸟醉芳春。",
    "白日登山望烽火，黄昏饮马傍交河。  \n行人刁斗风沙暗，公主琵琶幽怨多。  \n野云万里无城郭，雨雪纷纷连大漠。  \n胡雁哀鸣夜夜飞，胡儿眼泪双双落。  \n闻道玉门犹被遮，应将性命逐轻车。  \n年年战骨埋荒外，空见蒲桃入汉家。",
    "野客闲行满四郊，偶逢佳事辄迟回。  \n山禽忽起下空辙，水鹿应寒傍古台。  \n欲去未去常独立，自问无心得几来。  \n更想前溪好泉石，可能留得使君才。",
    "一别秦城万里遥，五更秋月九天高。  \n露侵驼褐浑无奈，霜入貂裘亦自牢。  \n宿鸟声危翻夕照，归人影断隔寒潮。  \n瞿塘峡口曲江头，万里风烟接素秋。",
    "欲问西江近消息，小姑洲上足离忧。  \n不堪花落南山望，更逐东风过石头。",
    "年华共寂寞，长在后园中。",
    "三月正当三十日，风光别我苦西东。  \n自怜两鬓清霜早，又见花时一病翁。",
    "独坐南窗下 弹琴拟自欢 岂无怀归恋 更有苦身难 羽翼皆摧落 风尘独浩漫 虽当鸣凤集 不肯暂相看",
    "几宿北林下，今朝南陌春。还持千日醉，共作百年人。日月东西驶，山河表里新。何因却归去，江海有遗尘。",
    "万里别君期，相逢还几时。  \n离堂酒未醒，征棹动明水。  \n日落大荒西，苍茫平不夷。  \n故人不可见，肠断秋风归。",
    "万里经年别，孤村此夜愁。 清灯独不寐，风雨到更初。 梦觉疑连榻，啼休欲断钩。 那堪听鸣雁，一一到西州。",
    "百岁光阴半发时，此身行止信难期。  \n自怜白发非前镜，不觉青春属旧诗。  \n云水有心常独往，江湖无伴更相随。  \n近来渐喜经旬卧，唯拟将书就尔师。",
    "万国正纷争，中原有盛明。  \n不知天下士，皆道子云生。",
    "白日登山望烽火 黄昏饮马傍交河 星临玉井动太液 晓入银河闻佩环。",
    "独行潭底影，数息树边身。",
    "万里初辞汉，三年遂事戎。孤臣心向北，万里道还东。积雪明沙漠，群峰辨地形。年年上高处，未省不伤情。",
    "昔我未识子，谁令问穷通。平生竹林下，亦有几人同。晚岁方为客，相逢尽白头。因君惜流景，万里一长讴。",
    "自昔牧二州，分功不赋秦。____岑参  \n当时黄霸辈，亦拟置公卿。____刘禹锡  \n晚节渐于诗见爱，早年惟解数为生。____白居易  \n愿学长松作君伴，何因得似大夫贞。____白居易  \n三入承明庐，一麾沧海壖。____王安石  \n此行非不豫，前事定难任。____王安石  \n但恐班行内，相逢有后心。____王安石",
    "昔岁逢正月，山城隐匡霍。  \n今来值迁次，郡宇接金络。  \n方事东道交，且将南国约。  \n江分越王泪，人传楚客作。  \n落日孤舟去，离忧万里泊。",
    "白日斜时酒易醒，绿芜墙外别人耕。  \n不知何处得秋气，明日黄花插满瓶。",
    "昔岁逢杨叛^{①}，今秋佐汉兵。无令探虎^{②}子，辛苦向辽城。",
    "昔在洛阳时 亲友同舟发 长江几千里 唯见行帆没 晚泊浔阳渚 月明闻棹歌 君门一不见 南望但烟波",
    "不独君恩许远游，亦将岁月付归舟。  \n故山未有开耕信，犹得相将过白头。",
    "曾逐东风拂舞筵。乐游春苑断肠天。 如何肯到清秋日，已带斜阳又带蝉。",
    "曾向瑶池醉日边 紫泥斜覆紫云笺  \n自怜尘土无因到 便恐烟霞有分缘",
    "百国千城闭不开，将军西向领兵回。  \n边烽不举河源静，虏骑无多塞北来。  \n万里山川将客老，五更刁斗逐人催。  \n如何共结东归计，却拟移家向海云。",
    "未向人间识密机 先从天上去多时  \n几回南陌怜芳草 今日西楼看落晖",
    "不作河西尉 无由见玉关",
    "天宝兵戈满帝乡，建溪新贡救年荒。  \n不知何处飞来燕，衔得朝烟入洞房。",
    "一雨三日行不得，诸公千里起相迎。  \n如何未到家犹喜，却恐家贫却不能。",
    "独有东归客，相逢亦式微。山连越国尽，海接故乡稀。晚泊鱼行近，秋风雁影微。不堪回首望，衰鬓已成丝。",
    "春草年年绿 朝云日日愁  \n人间更有病 一夕是归秋",
    "自昔怀清赏 陶然卧北林 因高偶动兴 千里旷平阴 落叶飘寒砌 长云度夕岑 风鸣北窗里 月映南池心 岂不厌兹苦 惟君适我寻",
    "丹青画长白兮已老，玉版写西真兮难保。  \n愿餐金液及刀圭，不问人间千岁药。",
    "独有青袍尉，能知白雪音。逢山皆解带，入水不通砧。地僻人稀到，朝回吏自寻。谁怜汉庭老，犹卧茂陵阴。",
    "画鼓娇初动，春旗喜尽褰。雪英飞一曲，风叶下三天。独有怜才子，能令辍管弦。自怜花下客，长听侍臣篇。",
    "曾将玉女镜台前。却向人间别作妍。  \n应是临邛双女妒，却教飞去著云边。",
    "昔往由来地，今登旦暮台。人悲历乱后，鸟倦寂寥来。日落西山外，天高北固开。独怜清渭水，东注此中回。",
    "自昔怀清赏，神游杳霭间。何须穷水石，始得问江山。地接长沙近，人当郢县闲。扁舟何太晚，吾亦倦飞还。",
    "汉室昔云季 七雄各尊王 荆舒遂僭拟 魏晋无纲常 殷鉴谅不远 何由观未光",
    "天际识归舟 窗中见远岫 无因驻衰老 更得奉交游",
    "万灶貔貅夜不惊，天戈指日静欃星。  \n汉家旄节新从北，胡地桑麻旧有名。  \n万里河倾落日精，三边气肃上皇清。  \n玉关西向犹多事，金印悬知合有盟。",
    "天作高山谁得似 气弄清风自好音  \n瑶草有情皆带恨 碧桃无主亦含愁",
    "海气侵陵日半曛，黄埃漠漠草茫茫。  \n野人相见无时节，行处唯闻马上声。",
    "万国尚戎旅，故园今若何。空馀天禄阁，高对建章宫。日落塞云合，水寒沙路空。汉家无旧老，谁见茂陵翁。",
    "汉寝唐陵无麦饭，山溪野径有梨花。  \n一朝哭都市，前后死人接。",
    "万国仰神京。三川镇北平。山河天故在。京洛人俱行。地拥千门入。途分九陌横。风烟今已远。冠盖日方并。夹路徒言好。经游讵可胜。徒令望归叹。更觉此生轻。",
    "骨瘦面多尘，行逢又一春。不知何所用，白尽少年人。",
    "何以娱晚景 东皋有故园 岂无平斗地 新有短辕门 醉卧黄公店 醒登青岳原 眼明逢酒伴 足力得柴孙 欲识忘机久 心随飞鸟翻",
    "昔岁逢嘉节，今兹对病身。不堪重倚帽，未暇整经纶。旧馆空台殿，新霜满缙绅。西园有遗树，零落共谁春。",
    "病身无定处，秋气入枯肠。日月长于人，寒暄暗自强。病来知世薄，老去觉心降。何以慰幽独，强将诗酒当。",
    "赤帝有灵应识我，女床无树不啼猿。  \n当时若更逢黄石，未用烦君说数难。",
    "玉井晨光动，金房宿雾披。开时幽鸟下，落处远人迷。应有仙桃树，年年自作蹊。"]
main_result = grade(poem)
print(main_result)

['3 1 1 2 3', '4 5 5 4 8', '4, 4, 4, 4, 7', '请提供第二首诗歌。', '①格式：2  \n②创新性：4  \n③相关性：5  \n④美学：3  \n⑤整体：5', '①格式：4  \n②创新性：4  \n③相关性：4  \n④美学：4  \n⑤整体：7', '①格式：0\n②创新性：0\n③相关性：0\n④美学：0\n⑤整体：0', '3 5 5 4 7', '4 4 5 4 7', '4, 4, 5, 4, 8', '①格式：2  \n②创新性：3  \n③相关性：4  \n④美学：3  \n⑤整体：5', '4, 4, 5, 4, 8', '4 5 5 4 8', '4, 4, 4, 4, 7', '5 0 5 5 10', '①格式：1  \n②创新性：4  \n③相关性：3  \n④美学：2  \n⑤整体：4', '①格式：4  \n②创新性：4  \n③相关性：5  \n④美学：4  \n⑤整体：7', '4 4 5 4 8', '4，5，5，4，8', '①格式：4  \n②创新性：4  \n③相关性：5  \n④美学：4  \n⑤整体：7', '4 4 5 4 8', '4 4 5 4 8', '4 5 5 4 8', '5 5 5 4 8', '3，4，4，4，7', '①格式：2  \n②创新性：4  \n③相关性：3  \n④美学：3  \n⑤整体：6', '4，5，5，5，9', '请给出两首诗歌，以便我进行评价。', '3 5 4 4 7', '① 3  \n② 4  \n③ 4  \n④ 3  \n⑤ 6', '4 5 5 4 8', '①格式：3  \n②创新性：4  \n③相关性：5  \n④美学：4  \n⑤整体：6  ', '格式：5  \n创新性：4  \n相关性：5  \n美学：4  \n整体：8', '4，5，4，4，8', '4 5 5 4 8', '①格式：2\n②创新性：4\n③相关性：4\n④美学：3\n⑤整体：5', '①1 ②3 ③4 ④2 ⑤4', '①格式：2  \n②创新性：3  \n③相关性：4  \n④美学：3  \n⑤整体：5', '4 5 5 4 8', '①格式：4  \n②创新性：4  \n③相关性：4  \n④美学：4  \n⑤整体：7', '4, 3

In [ ]:
def loading(data_location, generation):
  with open(data_location, "w", encoding="utf-8") as f:
      json.dump(generation, f, ensure_ascii=False, indent=4)
  return 0

In [ ]:
loading("Baseline Generation.json", poem)

0

In [ ]:
data = ['春风吹动青丝发，  \n芳草萋萋绿满园。  \n少年如花娇艳丽，  \n笑靥如花动人魂。  \n\n梦想披荆斩棘路，  \n热血沸腾勇向前。  \n岁月匆匆逝如箭，  \n青春永驻心头田。', '梦中花开映月华，\n一念飞扬入云霞。\n翩翩羽翼逐风起，\n逍遥天地逸尘沙。\n\n凌波微步登仙境，\n丹青妙手绘神华。\n若问梦想何所在，\n心中自有长安花。', '青峰耸立云霄间，\n巍峨高山入画图。\n挺拔峰峦云雾里，\n千古传颂壮丽愈。\n\n巨石奇峰立云天，\n峻岭奇峰耸碧烟。\n苍松翠柏摇风韵，\n山水相连意气鲜。', '深海茫茫波浪阔，\n浮光跃金鳞翩跹。\n深情莫测似幽梦，\n浩渺无涯藏玄机。\n\n潜游鲸鲵动轻盈，\n涌泉飞珠溅涟漪。\n蓝色苍茫心荡漾，\n谁能洞悉其中味。', '繁星点点银河横，\n夜幕降临掩昼光。\n寂静宇宙无边际，\n漫天繁星耀眼中。\n\n流萤点点飞舞间，\n思绪如云难安宁。\n仰望苍穹心悸动，\n星空璀璨梦中行。', '寂寞深夜失落情，\n孤灯孤枕泪两行。\n忆往昔恩爱如水，\n如今空留断肠声。\n\n风吹残梦入悲伤，\n心事难诉悔难当。\n花落花开皆匆匆，\n人间离合总难长。', '红颜昔日共谁知，\n相携并肩心相随。\n风吹花落泪珠滴，\n情深意重恨无疑。\n\n山水无情情自长，\n爱恨情仇皆虚妄。\n但愿相伴到白头，\n此生此世情不凉。', '独立寒江水，孤舟泛夕阳。\n寂寞思无限，孤灯照空房。\n独坐听风雨，孤云随飘忙。\n孤独何所恃，唯有心声长。', '明月高悬挂枝头，\n繁星点缀夜空愁。\n独倚栏杆思无尽，\n只愿人间皆如初。', '迷茫心绪难平静，烟雨蒙蒙映古城。\n柳岸花间听杜鹃，思绪飘飘恍如梦。\n人生若只如初见，何须纷扰乱心弦。\n行云流水任往来，迷茫追寻自心间。', '青春岁月似流水，成长之路漫漫长。\n蓦然回首思往事，曾是稚嫩少年郎。\n勤奋苦学逐梦想，携手挑战风浪涛。\n历经坎坷锤炼心，方知人生不易忘。\n\n青山不老绿还青，成长之路自有情。\n莫待时光匆匆逝，珍惜当下莫虚耽。\n扬帆远航逐梦想，勇往直前无畏惶。\n终将功成名不负，青春如歌笑颜开。', '暮色苍茫忆故人，离别情长泪沾巾。\n相思万缕难言语，断肠人在远山春。', '碧水横流共白鹭，重逢之际泪交绪。\n前尘往事随风散，今日相见喜悦无。\n山川万里皆有情，相逢何惧岁月残。\n且将心事随风散，重逢之际共欢颜。', '夕阳西下悠悠，\n余晖映照山丘。\n红霞满天飘渺，\n暮色笼罩乡愁。\n\n林间鸟啼声断，\n池畔花影摇摇。\n风吹轻拂鬓发，\n思绪如云飘飘。\n\n归路漫漫无尽，\n心事牵绊如絮。\n独坐斜阳之下，\n泪湿相思之袖。\n\n愿得此情长留，\n与君共赏晚愁。\n待到明月当空，\n共话人生忧愁。', '晨曦初现照乾坤，霞光初现拂山峦。\n鸟啼声声惊梦觉，花香飘飘入帘栊。\n晨风拂面心悠然，一日之计在于晨。\n世事纷扰皆抛却，清晨一刻身心宽。', '白雪纷纷起，寒风凛冽来，\n林间树梢挂，玉枝披银盖。\n寂静无声夜，万籁皆寂寥，\n瑞雪如花落，铺洒银装地。\n\n山川皆银妆，江海一片白，\n冰清玉洁美，寒气逼人怀。\n瑶池飘素雪，仙乐韵悠哉，\n冬日寒雪至，心中温暖来。', '春风吹拂花枝舞，翩翩绽放绚烂花。\n芬芳馥郁满园林，花开花落总相依。\n春日暖阳照人心，花间蜂蝶舞翩翩。\n红芳白艳争妍丽，春花开放赏芳菲。', '夏雨轻飘落，滋润大地间，\n翠竹摇风起，花香溢园园。\n微雨如丝洒，拂去尘埃烦，\n清风吹窗外，柳岸翠意悠远。\n鸟语声声动，蛙鸣夜夜闻，\n思念如泉流，永恒心间存。', '秋叶飘零落，寒风凄冷切。\n一树残红翠，思绪随风追。\n凄凉入诗骨，怀念如潮涌。\n落叶知多少，离人泪满衣。', '往事如烟逝，\n悠悠岁月情。\n昔日少年心，\n如梦入幽冥。\n\n桃花笑春风，\n琴瑟韵离声。\n几度芳华过，\n今朝已成尘。\n\n故人何处去，\n烛火寒星明。\n唯有心中梦，\n永留在我生。', '昔日欢乐今何在，\n忧伤萦绕心头来。\n风吹花落思无穷，\n泪洒残阳天地灰。\n\n梦境虚幻如烟波，\n凄凉寂寞难招魂。\n寒夜孤灯照空房，\n忧伤无尽恨情伤。', '欢乐自古情何限，笑颜如花乐意甜。\n欢歌笑语轻风起，欢乐盈盈心欢然。\n灯火辉煌照人间，欢乐时光醉美间。\n且将欢乐常相伴，共享快乐乐无边。', '迢迢行万里，\n拾遗探古迹。\n山川皆有情，\n风雨共徘徊。\n\n独立观星斗，\n孤舟泛清波。\n行尽天涯处，\n归来仍自我。', '迢迢归路漫漫长，\n孤舟泛水望乡忙。\n烟雨落花情未了，\n归心如梦梦初央。\n离人泪滴湿衣襟，\n故国风情入梦魂。\n愿将此生托清梦，\n终得归宿临窗前。', '山水相逢似故人，\n相谈甚欢共饮春。\n风吹笑语入诗意，\n月下歌声动心神。\n情深意长如长夜，\n友谊相伴似流云。\n世事多艰难易过，\n唯有朋友共同闯。', '五世同堂情意浓，百年传承在此中。\n父母慈爱如山川，兄弟姐妹情更浓。\n团圆时刻喜洋洋，别离时刻泪潸潸。\n家庭温馨如春水，幸福美满似云烟。', '古诗：\n\n传统何其美，历久弥新情。\n古训传家久，智慧代代传。\n丝绸织成衣，礼乐传后人。\n文化流芳远，风华永不倦。', '古代诗歌的格式通常为五言或七言绝句，我将为您写一首七言绝句：\n\n新意翻飞入古乐，创新心绪绘新篇。\n千载传世情何限，一朝开启梦想船。', '翩翩白鹭舞高天，  \n自由飘逸在云间。  \n山川河流皆我家，  \n心随风逐浪潮间。  \n\n世事纷扰不足忧，  \n心中自有一方宁。  \n任意飞翔逍遥处，  \n任凭花开叶自新。  ', '题目：约束\n\n铁索环绕心头缠，\n束缚思绪入牢笼。\n自由之风何处寻，\n囚禁身心两难容。\n\n愿破枷解锁心灵，\n解除羁绊重获翁。\n任风吹散束缚丝，\n自由飞翔梦中空。', '城市高楼耸入云，\n车水马龙忙碌身。\n繁华喧嚣人潮涌，\n梦想之地谁不羡。\n\n灯火璀璨夜未央，\n霓虹闪烁似星光。\n千家万户灯火亮，\n繁星点点似宝璋。\n\n商贾云集市井深，\n谋生谋利皆心恒。\n风云变幻谁能料，\n红尘滚滚难停腾。\n\n城市喧嚣又静谧，\n梦想舞台任我游。\n饮尽繁华心自定，\n一身诗意自飘逸。', '炊烟袅袅映村庄，\n稻菽飘香满小庭。\n樵夫归来歌远道，\n麦田稻海耕者忙。\n林间溪畔闲徜徉，\n鸟语花香入梦乡。\n岁月静好乡村景，\n心怀感慨情无疆。', '悠悠河水绕林间，\n流淌不息入海潮。\n涓涓涤荡洗尘埃，\n承载着岁月的沧桑。\n\n波光粼粼照花影，\n草长莺飞映画桥。\n行舟自在心自闲，\n任风吹拂任浪涛。\n\n忽起涟漪心悸，\n河水如歌吟低回。\n岁月如梭流不尽，\n唯愿平淡似水来。', '山谷幽深古老情，\n翠峦绵延抱清泉。\n晨曦初现烟雾敛，\n夕阳西下影斜绵。\n猿啼鸟语声相和，\n风吹松涛意自然。\n人间喧嚣皆抛却，\n心随山谷入梦烟。', '黄沙漫漫似金砂，孤鹰翱翔掠残花。\n独立荒漠心自苦，万木凋零泪满沙。\n烈日炎炎照天地，寂寞无人共孤华。\n但愿风雨洗尘埃，再现昔日绿林花。', '林间幽径隐飞花，  \n翠影婆娑萦古槐。  \n清风拂面吹细雨，  \n寂寞时光度春来。  \n松间鸟语韵琴韵，  \n溪畔花开如画开。  \n愁人心事随流水，  \n唯愿时光不匆匆。', '月华如水洒银池，寂静夜色入梦思。\n繁星点缀空中澄，微风拂过古松吟。\n夜深人静思徘徊，思念缠绕心头回。\n繁华尽处皆空寂，唯有夜色伴孤灯。', '朝阳初升照大地，\n黎明时分梦醒来。\n露华浓，云淡轻，\n一缕曙光照窗外。\n\n鸟儿啁啾报晓声，\n山川苏醒迎朝晖。\n花开半掩眠衣冷，\n清风拂面心意追。\n\n黎明光影入怀袖，\n万籁寂静思无妨。\n红日初升照人心，\n岁月如歌梦依稀。', '细雨萧萧映寒烟，\n林间幽鸟啼寂寥。\n阴云密布遮天色，\n江畔柳枝垂泪摇。\n\n愁思如丝缠心扉，\n悠悠岁月泛情怀。\n忧伤何处寻解脱，\n只见阴雨滴寒潮。', '碧空如洗晴日明，\n微风拂面心情轻。\n翠竹含翠迎朝露，\n红梅报春舞晓晴。\n山川浩瀚云水碧，\n禽鸟啼鸣林影清。\n万物生长皆有象，\n晴天万里心自惊。', '翠峰葱笼拥碧湖，\n红叶飘零舞翩跹。\n岸边渔火烟波暖，\n晨曦初现水天间。\n\n山川起伏如画意，\n风吹细雨润花田。\n诗意盎然心自醉，\n世外桃源在眼前。', '人群熙熙拥挤行，\n喧嚣声中情自生。\n相遇相知皆匆匆，\n一瞥一笑心相通。\n\n繁华世界似梦境，\n喧嚣尘世终虚空。\n人群中寻一知己，\n相携共赏明月中。', '独坐幽篁里，\n寂寞无人知。\n林间鸟啼处，\n孤影独徘徊。\n\n风吹落叶下，\n寒日照空枝。\n独立江边望，\n孤帆远影垂。\n\n心事随风远，\n烛影摇微微。\n孤灯对夜色，\n孤魂泪漫飞。', '喧闹喜悦逐风来，\n繁华都市热闹开。\n商贩叫卖声声动，\n游人笑语处处来。\n\n灯火辉煌映夜色，\n繁星点点耀金台。\n酒香飘散花园里，\n欢歌笑语绕耳畔。\n\n热闹场面喜气洋，\n丝竹管弦奏乐章。\n心中情意难言表，\n热闹喧嚣尽欢畅。', '湖面波澜平，水天一色静。\n微风吹过岸，草木摇曳轻。\n心如湖水静，思绪渐平衡。\n任白云飘散，任时光悠然。', '烈火燃烧激情生，\n心中热血涌泉涌。\n愿与君共赏春花，\n共谱壮丽诗篇。\n激情如炬照人生，\n奋发图强志气凌。\n磅礴豪情犹未尽，\n梦想何时能成真。', '千山压顶势汹汹，\n忧虑纠缠意难宁。\n心如乱麻难解绕，\n忧愁萦绕梦中行。\n\n日升日落忧思在，\n苦恼煎熬似火烹。\n一念之间如波澜，\n何时能得心宁静。\n\n浮生苦海途漫漫，\n压力如山难披荆。\n但愿坚守心中梦，\n逆水行舟不回头。', '长夜漫漫无尽期，苦海波涛难解脱。\n迷途羁绊心已乱，何处才能得安宁。\n苦心孤影难相伴，苍天无情泪两流。\n唯愿忧愁皆成空，解脱纷乱入梦乡。', '追寻古道寻真我，  \n步步寻觅在人世。  \n山川湖海皆留迹，  \n风雨雷电共徜徉。  \n\n夜深人静寻星斗，  \n烛火微微照古城。  \n问古问今心不改，  \n千古情思绵绵情。  ', '春风拂面发现新，\n\n山水交相映日辉。\n\n红梅傲雪迎春至，\n\n碧海蓝天梦幻时。\n\n探寻星河璀璨处，\n\n触摸人间美景悲。\n\n心怀感慨情何限，\n\n发现真我在诗境。', '花开花谢又一秋，\n人间离别几多愁。\n悠悠岁月如流水，\n失去何处寻心休。', '题目：明月如霜\n\n明月如霜，照耀人间。\n银光洒落，寂静安详。\n流连忘返，思绪万千。\n挂在枝头，似水中天。\n\n皎洁如玉，清辉璀璨。\n孤影寥落，寄情何处。\n想起故人，泪湿衣襟。\n共赏月色，共话心事。\n\n明月如霜，皎洁无瑕。\n世事沧桑，岁月如花。\n但愿人间，皆安康和。\n与明月同，永无忧伤。', '往日时光转眼过，\n桃花依旧笑春风。\n青梅煮酒情何限，\n千年等待终成空。\n\n故人已去无踪迹，\n留下空留旧相思。\n泪洒相思情难断，\n唯愿来世再相见。', '晨曦初现新生芽，翠叶婆娑拂晓霞。\n轻风吹拂花香满，清泉潺潺润林花。\n青鸟啼鸣报春意，碧空万里展宏图。\n生机盎然天地间，新生希望满人间。', '翠竹寒风起，悲鸣入夜深。\n孤灯影摇曳，泪滴花枝阴。\n凄凉思无尽，情绪难安宁。\n忆往昔欢乐，今夜泪霑襟。', '欢歌笑语喜气生，\n春风拂面心情轻。\n花开花落皆如意，\n喜悦满怀乐意盈。\n\n万物皆欢喜我乐，\n福至心间喜洋洋。\n岁月静好如画境，\n喜悦常驻心头藏。', '离别伊人泪已干，  \n相思枕畔梦难安。  \n曾经誓言如今散，  \n一切皆成往事烟。  \n断桥残雪寒更深，  \n独立寒江泪满襟。  \n何须再问离恨苦，  \n且把离愁寄远岑。', '九州团聚乐未央，亲朋好友共欢颜。\n花开花落皆有期，团圆一梦情更浓。\n酒满金杯情意长，歌舞欢歌笑语长。\n团聚之时心相许，共话家国共抒怀。', '哲理虚无似清风，智慧如水润心灵。\n探究天地皆有理，琢磨人生岂无情。\n\n春风吹拂花已开，秋月皎洁思悠哉。\n行走江湖求真理，悟尽人间苦与乐。\n\n万千世界皆虚幻，唯有哲理永长存。\n修身养性须励志，心怀大道方得真。', '风起纷争正当时，\n世道沉浮事难知。\n剑指苍穹云烟起，\n心存浩然意气衰。\n\n谁言胜负在劫数，\n英雄何须问前程。\n只愿平安人心定，\n岁月如歌不复追。', '丹青画和平，世界共安宁。\n山河长明镜，风月共清庭。\n金戈铁马息，云水共和鸣。\n人间纷争尽，万古一乐生。', '江湖险恶路漫漫，\n剑指苍穹志未残。\n莫道江湖多艰险，\n只因豪气在胸藏。', '夜深人静思安全，独坐窗前心不安。\n灯火摇曳犹未息，寂静之中梦惊魂。\n\n路途漫长须谨慎，前行莫忘安全心。\n危机四伏需警惕，平安如意愿常存。\n\n家园固守才安稳，子孙后代得幸福。\n安全之道人皆晓，平安顺遂在其中。', '书山万仞耸苍穹，学海无涯探幽踪。\n秉烛夜读犹未倦，披星戴月谒先雄。\n铭心刻骨求真理，笃志笃行奋斗躬。\n功名利禄皆浮云，唯有学识是长存。', '教育如泉水，滋润心田育英才。\n言传身教德为先，开启智慧慧芬芳。\n书山有路勤为径，学海无涯志为航。\n春风吹拂百花艳，教育功成天下安。', '登高望远志，成就在心间，\n千山万水路，一步一挑战。\n勇攀峻岭上，俯瞰云天宽，\n风吹心中火，热血燃烧间。\n\n刀锋试磨砺，星光照征衣，\n不畏艰险险，奋勇向前行。\n功勋留青史，荣耀传万年，\n成就非止身，更是心灵航。', '曲径行人独彷徨，挫败之时梦断肠。\n功名利禄皆如梦，一朝失败泪沾裳。\n昔日豪杰今何在，落英缤纷随风扬。\n唯余寂寞思难奈，独自凭栏对残阳。', '红旗飘扬胜利来，\n金戈铁马战无停。\n万众同心齐奋发，\n星光熠熠照乾坤。\n\n风云变幻浴战场，\n英雄豪气决乾坤。\n虎啸龙吟声未断，\n胜利之路任吾行。', '乾坤变化竞风云，英雄豪杰展雄心。\n剑指苍穹争一席，斗艳红颜比翠鬓。\n争夺荣耀谁更胜，斗志昂扬不停深。\n唯有拼搏才成就，竞争之路任踏寻。', '古诗《合作》\n\n同舟共济路漫漫，相携并肩任征端。\n风雨同舟心相系，团结一心任重山。\n\n共商大计言欢笑，共谋伟业志更坚。\n携手同心齐努力，合作共赢梦向前。', '夜深梦境入幽冥，梦魇翩跹舞幽灵。\n惊魂夜夜常相扰，悲啼声声恐魅惊。\n黑暗笼罩心灵寂，晨曦初现梦终醒。\n回首往事空悔恨，梦魇何时能消净。', '幻想乘风起，梦境随云飘。\n世间纷扰尽，心中自由翔。\n山水皆可至，花香入梦遥。\n晨曦抚我面，星夜梦中朝。', '沧海桑田变古今，\n现实如梦难辨真。\n岁月匆匆人易老，\n梦想难寻心不宁。\n\n珠玉满堂空虚幻，\n名利场中苦劳神。\n唯愿珍惜眼前景，\n懂得知足才安宁。', '山高水远逃无踪，\n潮起潮落梦中空。\n逆风飞去心难定，\n逃避现实梦中逢。', '题目：秋风萧瑟\n\n枯叶飘零秋风起，萧瑟寒意入人心。\n寂寞凄凉何处寻，孤灯独影夜未央。\n\n思绪飘飞随风去，悠悠岁月如流水。\n凝望远方无尽处，泪湿衣襟叹离愁。\n\n时光荏苒转瞬间，人生如梦悲欢聚。\n挥袖摇曳任风吹，秋风萧瑟又一秋。', '山水变幻情何极，岁月更迭恍如梦。\n春风吹拂花枝翠，秋雨淅沥叶萧萧。\n人生若浮云飘渺，世事如流水悠悠。\n变化无常难预料，唯愿心境永安宁。', '恒常千古在人间，\n岁月流转不改变。\n春风秋雨皆如故，\n山河星辰皆静观。\n\n古木参天见证者，\n流水潺潺述沧桑。\n人生短暂如浮云，\n唯有恒常永长存。', '往事如烟逝，追忆心头转。\n昔日欢乐时，如今已成梦。\n忧伤随风起，泪水湿衣襟。\n唯愿时光慢，再见旧时人。', '岁月如梭逝，遗忘心头事。\n忧伤随风起，思念随风去。\n往事如梦迷，无处觅踪迹。\n唯愿时光静，遗忘烦恼迹。', '碧水悠悠映梦魂，\n古诗传世情无穷。\n墨翰轻挥思万千，\n诗意盈怀泪珠翻。\n\n风吹柳梢情意长，\n凝神款款思绪舒。\n笔下繁华如画卷，\n诗心流淌入江楼。\n\n诗韵悠扬入云端，\n心灵清净如琉璃。\n千古传承情意长，\n诗魂永驻在人心。', '画笔轻挥彩虹飞，\n墨色浓淡写人生。\n山水山川皆入画，\n一笔一划情无限。', '银簧金管奏佳音，\n琴瑟和鸣韵动人。\n悠扬旋律如泉流，\n律动心弦醉耳陈。\n\n琵琶箫管唤清风，\n弹指间间情已浓。\n音律跌宕情何限，\n音符飘逸入云中。\n\n醉卧瑶台听管弦，\n歌声婉转入梦烟。\n音乐声声入心扉，\n留下千古传千年。', '清风拂面舞纤腰，舞动轻盈似游鱼。\n芳草如茵春意暖，舞台繁花绽芳姿。\n\n手揽云霞倚碧海，足踏九天舞彩虹。\n舞罢犹存余音在，心中留下一缕香。\n\n舞尽春风又一曲，情意绵绵化流水。\n飘飘舞动如梦幻，人间仙子舞长衣。', '石砌山水立，玉凿精雕成。\n流光溢彩影，永恒定其形。\n凝神凌空望，心驰碧云横。\n千古传芳华，岁月留印生。', '高楼耸立入云端，石梁砖瓦凝英雄。\n千年风雨铸辉煌，万丈高楼逐梦想。\n玉宇琼楼耀神采，金瓯银樽赏春光。\n建筑巍峨凝古韵，传承文明万古长。', '青烛照书台，墨香入帙开。\n笔端舞文章，心灵抒离怀。\n纸上流飞翰，墨色浸春来。\n诗情荡人心，文学永流传。', '往事如烟逝，历史长河阔。\n一笔一画写，千秋万代传。\n英雄辈出时，谁主沉浮浪。\n铁骑铿锵过，留下世人叹。\n古今交相辉，兴衰盛衰间。\n岁月无情转，唯有历史留。', '未来展宏图，何处是归途。\n千山万水间，一路向前途。\n风吹云散处，星辰闪耀初。\n勇往直前行，胸怀万千愿。', '铁马行遍四方程，机关巧夺天工权。\n流光溢彩铸奇迹，智慧凝聚创未来。', '青山绿水映朝霞，\n碧波荡漾鱼翔跃。\n风吹柳絮舞轻盈，\n花开蝶舞恣情怀。\n\n浮云飘飘遮日月，\n流水潺潺语叮咛。\n山林幽深藏奇趣，\n自然造化真神奇。', '山崩水竭日月昏，烽烟四起恐人魂。\n灾难降临无处避，哀鸿遍野泪沾裙。', '乱世飘零几多年，恢复往日梦未圆。\n烽火燃尽山河老，英雄豪气再谁传。\n铁衣银甲埋尘土，破败残垣见风尘。\n独步长路悲歌起，恢复往昔心不言。', '江水东流尽，落日照斜阳，\n山河苍茫远，岁月如梭忙。\n世事无常定，衰退自可望，\n凋零非终局，新生在远方。', '繁荣一片耀金黄，\n万物生长展笑容。\n江河波涛潮水急，\n岁月流转景色浓。\n\n城市楼台耸云端，\n商贾云集市井忙。\n繁华背后藏玄机，\n富贵荣华难长久。\n\n人间繁荣如梦幻，\n唯有淡定守本心。\n不被荣华迷眼睛，\n心中清静方有真。', '荒野风吹衰草黄，\n寂寞孤影映残阳。\n往事如烟随风散，\n凝望远方泪沾裳。', '青山翠竹映朝阳，强健身心入画廊。\n拳脚矫健驰长空，锻炼精神展豪杰。\n挥汗如雨驱风雪，坚毅意志铸英豪。\n莫待时光匆匆逝，强健体魄永如春。', '疾病缠身苦无辰，医者良方未有因。\n白日昏昏睡不足，夜来哀哭梦频频。\n药石无情人莫识，病魔缠绕心如针。\n盼望早日疾痊愈，重获健康笑颜新。', '柳暗花明春水溢，疾病凋零治愈时。\n救人于危如拨云，温暖心扉润众生。\n医者仁心疗愈愈，一线希望照人心。\n九曲回肠消疾苦，光明曙色洗尘襟。', '古道传说流长久，\n山水之间有故事。\n银河耀眼闪烁处，\n传承千年不曾迷。\n\n神话传说久远存，\n凤凰展翅飞翔去。\n龙腾虎跃显威风，\n传承千古不曾绝。\n\n传说故事传心间，\n古往今来皆传承。\n仙山琼阁传佳话，\n传承万古不曾忘。', '神话流传几千年，  \n传说古老梦幻间。  \n仙山仙水远离尘，  \n神兽神人共天宴。  \n\n凤凰展翅舞华彩，  \n龙吟虎啸显威武。  \n琴瑟箫管奏清音，  \n仙女仙郎共欢乐。  \n\n神话故事千万卷，  \n铭记传承心不改。  \n世人追寻古之远，  \n神话永存神韵在。  ']

In [ ]:
loading("GPT-3.5-Turbo Generation.json", data)

0